# Differential association testing of GNPS annotated molecules

**Author**: Madeleine Ernst (mernst@ucsd.edu) <br>
**Edited by**: - <br>
**Use case**: Differential association testing of GNPS annotated molecules (binary data) across categorical and continuous metadata types. <br>
**Input file format**: <br>
<ul>
<li>**Feature table with metadata** (.csv) table with samples in rows and metadata and GNPS annotated molecules in columns. </li>
</ul>
**Outputs**: html tables showing significantly differentially abundant GNPS annotated molecules across continuous and categorical metadata, csv tables showing all pairs tested for differential association. <br>
**Dependencies**: R version 3.3.3 (2017-03-06) and libraries stringr_1.3.1, htmlwidgets_1.2, ggplot2_2.2.1, reshape2_1.4.3

load libraries

In [1]:
library(reshape2)
library(ggplot2)
library(htmlwidgets)
library(stringr)

load data

In [3]:
data <- read.csv("Tables_MS2/20180829_Immuno15Skin_FinalOccuranceTablewithMetadata.csv",check.names = F,stringsAsFactors = F)

In [4]:
head(data)

filename,unique_sample_ID,unique_sample_ID.1,ATTRIBUTE_sample_ID,ATTRIBUTE_Meds_number,ATTRIBUTE_prescribed_acetaminophen,ATTRIBUTE_prescribed_albuterol,ATTRIBUTE_prescribed_allopurinol,ATTRIBUTE_prescribed_amlodipine,ATTRIBUTE_prescribed_aspirin,⋯,Sulfamethizole,Timolol,Triphenyl phosphate,Tris(2-butoxyethyl) phosphate,Tyr Pro,Undecaethylene glycol,Sulfachloropyridazine,Sulfamethoxazole,Syringaldehyde,Syringic acid
AA3594_1_1_RA1_01_37666.mzXML,AA3594_1_1_RA1,AA3594_1_1,AA3594,22,0,0,1,0,0,⋯,0,0,0,0,1,0,1,0,0,1
AA3594_1_10_RA10_01_37671.mzXML,AA3594_1_10_RA10,AA3594_1_10,AA3594,22,0,0,1,0,0,⋯,0,0,0,0,0,0,1,1,1,0
AA3594_1_2_RA2_01_37667.mzXML,AA3594_1_2_RA2,AA3594_1_2,AA3594,22,0,0,1,0,0,⋯,0,0,0,0,1,0,1,0,1,1
AA3594_1_3_RA3_01_37668.mzXML,AA3594_1_3_RA3,AA3594_1_3,AA3594,22,0,0,1,0,0,⋯,0,0,0,0,1,0,1,0,1,1
AA3594_1_4_RA4_01_37669.mzXML,AA3594_1_4_RA4,AA3594_1_4,AA3594,22,0,0,1,0,0,⋯,0,0,0,0,1,0,1,0,1,1
AA3594_1_5_diluted_RC5_01_37659.mzXML,AA3594_1_5_diluted,AA3594_1_5,AA3594,22,0,0,1,0,0,⋯,0,0,0,0,0,0,0,0,0,0


specify number of metadata columns

In [9]:
data[1:10,120:123]

ATTRIBUTE_Sample_Location_General_Sub1_Text,ATTRIBUTE_Sample_Location_General_Sub2_Text,alpha-Cyclodextrin,alpha-Hexylcinnamaldehyde
Forehead,R_Forehead,0,0
Palm,L_Palm,0,0
Forehead,L_Forehead,0,0
Nose,R_Nose,0,0
Nose,L_Nose,0,0
Axillary,R_Axillary,0,0
Axillary,L_Axillary,0,0
Palm,R_Palm,0,0
Backhand,R_Backhand,0,0
Backhand,L_Backhand,0,0


In [12]:
nmetadata <- 121

define method used for multiple hypothesis testing ("holm", "hochberg", "hommel", "bonferroni", "BH", "BY","fdr", or "none")

In [13]:
pad_meth <- "fdr"

prepare data for analysis

In [14]:
metadata <- data[,1:nmetadata]

# separate out all categorical metadata
types <- split(colnames(metadata),sapply(metadata, function(x) paste(class(x), collapse=" ")))
l_types <- c()
for (i in 1:length(unlist(types))){
  if (length(which(is.na(unique(metadata[,colnames(metadata)==unlist(types)[i]]))))!=0){
    x <- unique(metadata[,colnames(metadata)==unlist(types)[i]])[-which(is.na(unique(metadata[,colnames(metadata)==unlist(types)[i]])))]
    l_types <- c(l_types,length(x))
  }
  else {
    l_types <- c(l_types,length(unique(metadata[,colnames(metadata)==unlist(types)[i]])))
  }
}
cattypes <- unlist(types)[-which(l_types>10|l_types==1)]
# separate out all categorical metadata

data <- data[,(nmetadata+1):ncol(data)]

## Test for differential expression across categorical metadata 

In [15]:
###################################
#  categorical vs. categorical    #
###################################

all <- list()
pval <- list()

for (i in 1:length(cattypes)){
    
  # Intervention Group vs. ions
  selcat <- cattypes[i]
  
  # count 1's
  sel <- cbind(metadata[,colnames(metadata)==selcat],data)
  subcat <- as.character(unique(metadata[,colnames(metadata)==selcat]))
  
  if(unique(selcat)=="Visit"){
    subcat <- subcat[-which(subcat=="14")]
  }
  
  subcatsums<-list()
  
  for (j in 1:length(subcat)){
    subcatsums[[j]] <- colSums(sel[which(sel[,1]==subcat[j]),-1])
  }
  
  output <- matrix(unlist(subcatsums), ncol = ncol(data), byrow = TRUE)
  colnames(output) <- colnames(data)
  rownames(output) <- subcat
  if (length(which(rowSums(output)==0))!=0){
    output <- output[-which(rowSums(output)==0),]
  }
  
  # count 0's
  subcatsums_neg <-list()
  sel_neg <- sel
  sel_neg[,-1] <- 1-(sel_neg[,-1])
  
  for (k in 1:length(subcat)){
    subcatsums_neg[[k]] <- colSums(sel_neg[which(sel_neg[,1]==subcat[k]),-1])
  }
  
  output_0 <- matrix(unlist(subcatsums_neg), ncol = ncol(data), byrow = TRUE)
  colnames(output_0) <- colnames(data)
  rownames(output_0) <- subcat
  if (length(which(rowSums(output_0)==0))!=0){
    output_0 <- output_0[-which(rowSums(output_0)==0),]
  }
  
  # perform Fisher's Exact Test
  out <- list()
  p_values <- c()
  
  for (m in 1:ncol(output)){
    TF <- cbind(output[,m],output_0[,m])
    p_values <- c(p_values,fisher.test(TF,workspace=10e+08,hybrid = T)$p.value)
    out[[m]] <- cbind(TF,colnames(output)[m])
  }
  
  pval[[i]] <- p_values
  all[[i]] <- out
  names(all[[i]]) <- colnames(output)
}

names(all) <- as.character(cattypes)

pval_corr <- p.adjust(unlist(pval),method=pad_meth)  
  
sum_cat <- data.frame(metadata_ID=rep(cattypes,each=ncol(data)),ms_feature=rep(colnames(data),length(cattypes)),p_value=pval_corr,
                      metadata_type="categorical",test="Fisher's Exact Test for Count Data",padjust=pad_meth)

Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace =

“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning

“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning

“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning

“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning

...

“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning

“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning

“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning

“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning

“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning message in fisher.test(TF, workspace = 1e+09, hybrid = T):
“'hybrid' is ignored for a 2 x 2 table”Warning

define p-value cutoff: length(n) shows how many pairs will be retrieved with the defined cutoff

In [16]:
n <- which(sum_cat$p_value < 0.00000000005)
length(n)

[1] 98

In [ ]:
data$Lauric Diethanolamide

In [54]:
# create selected plots
mID <- "ATTRIBUTE_Sample_Location_General_Text"
f <- "Acetyl-sulfamethoxazole"

s <- all[[which(names(all)==mID)]]
s <- s[[which(names(s)==f)]]
colnames(s) <- c("present","absent","ms_feature")

if(length(which(is.na(rownames(s)))) != 0){
  rownames(s)[is.na(rownames(s))] <- "NA"
}

x <- as.data.frame(s[,1:2])
x$present <- as.numeric(as.character(x$present))
x$absent <- as.numeric(as.character(x$absent))
x$groups <- rownames(x)

df1 <- melt(x,id.var="groups")
df1$value <- as.numeric(as.character(df1$value)) 
df1$total <- ave(df1$value, df1$groups, FUN = sum)
df1$percentage <- (df1$value/df1$total)*100
df1$labels <- paste("n=",df1$total,sep="")
df1$newgr = str_wrap(df1$groups, width = 15)
df1 <- df1[df1$variable=="present",]
file=paste(gsub("[.]","",mID), gsub("[[:punct:][:space:]]", "", f), ".pdf", sep="")
if (nchar(file) > 50){
  file = paste(substr(file, 1,40),".pdf",sep="")
  pdf(file=file, width=2.5, height=2.5)
  p <- ggplot(df1, aes(x = newgr, y = percentage, fill = variable)) + 
        geom_bar(stat = "identity") + 
        labs(x = mID,y= "% of samples") + 
        geom_text(aes(label = labels), size = 2, hjust = 0.5,vjust=-0.5) + 
        theme(panel.background = element_rect(fill="white"),
                  panel.grid.major=element_blank(),
                  panel.grid.minor=element_blank(),
                  axis.ticks=element_line(colour ="black",size=0.5, linetype="solid"),
                  axis.text=element_text(size=6),
                  axis.text.x=element_text(angle=0, hjust=0.5, vjust=1.25),
                  axis.line=element_line(colour="black",size=0.5, linetype="solid"),                  
                  axis.title=element_text(size=6),
                  #aspect.ratio=1,
                  legend.position="none",
                  legend.title=element_blank()
            )
  print(p)
  dev.off()
} else {
  pdf(file=file, width=2.5, height=2.5)
    p <- ggplot(df1, aes(x = newgr, y = percentage, fill = variable)) + 
      geom_bar(stat = "identity") + 
        labs(x = mID,y= "% of samples") + 
      geom_text(aes(label = labels), size = 2, hjust = 0.5,vjust=-0.5) + 
      theme(panel.background = element_rect(fill="white"),
                  panel.grid.major=element_blank(),
                  panel.grid.minor=element_blank(),
                  axis.ticks=element_line(colour ="black",size=0.5, linetype="solid"),
                  axis.text=element_text(size=6),
                  axis.text.x=element_text(angle=0, hjust=0.5, vjust=1.25),
                  axis.line=element_line(colour="black",size=0.5, linetype="solid"),                  
                  axis.title=element_text(size=6),
                  #aspect.ratio=1,
                  legend.position="none",
                  legend.title=element_blank()
            )
    print(p)
  dev.off()
}

png 
  2

In [32]:

#######################

mypath_cat <- c()

for (i in 1:length(n)){
  mID <- as.character(sum_cat[n[i],1])
  f <- as.character(sum_cat[n[i],2])
  
  s <- all[[which(names(all)==mID)]]
  s <- s[[which(names(s)==f)]]
  colnames(s) <- c("present","absent","ms_feature")
  
  if(length(which(is.na(rownames(s)))) != 0){
    rownames(s)[is.na(rownames(s))] <- "NA"
  }
  
  x <- as.data.frame(s[,1:2])
  x$present <- as.numeric(as.character(x$present))
  x$absent <- as.numeric(as.character(x$absent))
  x$groups <- rownames(x)

  df1 <- melt(x,id.var="groups")
  df1$value <- as.numeric(as.character(df1$value)) 
  df1$total <- ave(df1$value, df1$groups, FUN = sum)
  df1$percentage <- (df1$value/df1$total)*100
  df1$labels <- paste("n=",df1$total,sep="")
  df1$newgr = str_wrap(df1$groups, width = 15)
  df1 <- df1[df1$variable=="present",]
  file=paste(gsub("[.]","",mID), gsub("[[:punct:][:space:]]", "", f), ".pdf", sep="")
  if (nchar(file) > 50){
    file = paste(substr(file, 1,20),".pdf",sep="")
    pdf(file=file, width=3, height=3)
    p <- ggplot(df1, aes(x = newgr, y = percentage, fill = variable)) + geom_bar(stat = "identity") + labs(x = mID,y= "% of samples") + geom_text(aes(label = labels), size = 1, hjust = 0.5,vjust=-1.5) + theme(text = element_text(size=4),axis.title.y = element_text(size=5), axis.text.y =element_text(size=5), axis.text.x = element_text(angle = 0, hjust = 0.5),legend.position="none")
    print(p)
    dev.off()
    mypath_cat <- c(mypath_cat,file)
  } else {
    pdf(file=file, width=2.5, height=2.5)
    p <- ggplot(df1, aes(x = newgr, y = percentage, fill = variable)) + 
      geom_bar(stat = "identity") + labs(x = mID,y= "% of samples") + 
      geom_text(aes(label = labels), size = 1, hjust = 0.5,vjust=-1.5) + 
      theme(panel.background = element_rect(fill="white"),
                  panel.grid.major=element_blank(),
                  panel.grid.minor=element_blank(),
                  axis.ticks=element_line(colour ="black",size=0.5, linetype="solid"),
                  axis.text=element_text(size=6),
                  axis.text.x=element_text(angle=0, hjust=0.5, vjust=1.25),
                  axis.line=element_line(colour="black",size=0.5, linetype="solid"),                  
                  axis.title=element_text(size=6),
                  #aspect.ratio=1,
                  legend.position="none",
                  legend.title=element_blank()
            )
    print(p)
    dev.off()
    mypath_cat <- c(mypath_cat,file)
  }
} 

sum_cat$plot <- "none"
sum_cat$plot[n] <- paste0('<a href=\"',"file://",mypath_cat,'">plot</a>')

write.csv(sum_cat, "DifferentialassociationTesting_MS2/SummaryTable_categorical.csv",quote=F)

# extract only features with plots (significant p-value)
dt_cat <- DT::datatable(sum_cat[n,], selection =  list( mode = 'single'),escape=F)
saveWidget(dt_cat, file="SummaryTable_categorical.html")

In [26]:
write.table(sum_cat, "DifferentialassociationTesting_MS2/SummaryTable_categorical.txt", sep="\t")

## Test for differential expression across continuous metadata 

In [21]:
###################################
#  categorical vs. continuous     #
###################################

# separate out all continuous metadata
cont <- types$numeric 
# separate out all continuous metadata

data_cont <- data
if (length(which(colSums(data_cont)==0))!=0){
  data_cont <- data_cont[,-which(colSums(data_cont)==0)] # remove ions present in none of the samples
}
data_cont <- data_cont[,-which(colSums(data_cont)<10)] # remove ions present in less than 10 samples

p_values_cont_sum <- c(rep(0,ncol(data_cont)))

for (i in 1:length(cont)){
  # Intervention Group vs. ions
  selcat <- cont[i]
  
  # count 1's
  sel <- cbind(metadata[,colnames(metadata)==selcat],data_cont)
  sel[,2:ncol(sel)] <- lapply(sel[,2:ncol(sel)], factor)
  p_values_cont <- c()
  
  for (j in 2:ncol(sel)){
    if(length(unique(sel[which(sel[,j]==1),1]))==1|length(unique(sel[which(sel[,j]==0),1]))==1){
      p_values_cont <- c(p_values_cont,"all observations are in the same group")
    } else {
      p_values_cont <- c(p_values_cont,kruskal.test(sel[,1] ~ sel[,j], data = sel)$p.value)
    }
  }
  p_values_cont_sum <- rbind(p_values_cont_sum, p_values_cont)
}

p_values_cont_sum <- p_values_cont_sum[-1,]
p_values_cont_sum[p_values_cont_sum=="all observations are in the same group"] <- NA
pvalues_adj <- p.adjust(as.vector(p_values_cont_sum),method=pad_meth)

p.adjust(as.vector(p_values_cont_sum[-which(is.na(p_values_cont_sum))]),method=pad_meth)
p_values_cont_sum_corr <- matrix(pvalues_adj,nrow=nrow(p_values_cont_sum),ncol=ncol(p_values_cont_sum))

rownames(p_values_cont_sum_corr) <- cont
colnames(p_values_cont_sum_corr) <- colnames(data_cont)

sum_cont <- melt(p_values_cont_sum_corr)
colnames(sum_cont) <- c("metadata_ID","ms_feature","p_value")
sum_cont$metadata_type <- "continuous"
sum_cont$test <- "Kruskal-Wallis rank sum test"
sum_cont$padjust <- pad_meth

[1] 7.738141e-01 9.029180e-01 8.530275e-01 7.558959e-01 1.662897e-01
  [6] 4.554701e-02 8.709233e-01 2.362620e-01 8.454135e-01 3.517187e-01
 [11] 7.143430e-01 1.987363e-01 5.597545e-01 1.324357e-01 4.554701e-02
 [16] 7.633735e-02 2.978189e-04 8.463109e-01 1.157600e-01 2.868294e-01
 [21] 7.143430e-01 6.187346e-01 1.283873e-01 1.984309e-05 3.103984e-05
 [26] 3.593591e-02 9.691538e-01 7.143430e-01 1.283873e-01 7.738141e-01
 [31] 3.121647e-01 9.174900e-02 2.958308e-02 2.139246e-02 7.143430e-01
 [36] 9.405419e-01 8.463109e-01 5.659931e-01 1.152552e-01 9.397298e-03
 [41] 6.566366e-03 5.659931e-01 4.202099e-01 9.214377e-01 9.146310e-01
 [46] 4.714908e-01 8.061228e-01 7.312505e-01 4.570643e-03 4.570643e-03
 [51] 2.325147e-05 1.228763e-05 6.004227e-01 8.002346e-01 9.510917e-01
 [56] 7.119587e-01 8.263821e-01 9.029180e-01 7.385993e-01 4.203985e-01
 [61] 1.318722e-02 1.179944e-01 2.976282e-01 9.472925e-03 2.031216e-02
 [66] 5.597545e-01 3.500161e-04 5.597545e-01 9.391941e-01 1.812536e-01
 [71] 2.376760e-02 5.494725e-01 6.949184e-03 4.924858e-01 3.017798e-01
 [76] 9.472925e-03 3.088256e-01 1.431907e-01 1.292950e-01 1.191563e-01
 [81] 6.330223e-01 8.454135e-01 3.067545e-01 8.454135e-01 1.324357e-01
 [86] 2.595899e-01 6.450831e-01 6.330223e-01 7.512253e-01 5.456159e-02
 [91] 8.454135e-01 1.276789e-01 3.017798e-01 5.862159e-01 5.862159e-01
 [96] 7.419157e-01 8.463109e-01 5.659931e-01 6.091637e-01 9.510917e-01
[101] 3.834093e-02 7.591309e-01 3.674885e-01 7.385993e-01 9.029180e-01
[106] 7.538326e-01 7.468609e-01 1.779348e-01 4.714908e-01 9.405419e-01
[111] 7.345416e-01 1.154314e-03 9.458067e-08 5.659931e-01 1.568741e-02
[116] 4.980101e-01 7.550711e-01 1.060964e-01 2.091344e-03 9.902980e-01
[121] 5.048138e-02 8.722588e-02 4.033944e-02 2.543515e-01 4.488489e-02
[126] 3.649643e-01 3.485337e-01 1.625547e-01 3.834093e-02 4.680508e-02
[131] 9.506995e-02 3.804685e-02 9.108910e-01 4.014384e-05 4.412563e-09
[136] 2.208971e-02 6.349373e-01 7.929615e-01 8.656084e-01 1.965181e-03
[141] 9.146310e-01 2.647416e-02 8.036937e-01 1.669644e-05 2.382652e-05
[146] 4.820840e-01 1.568741e-02 5.089094e-04 2.567175e-02 6.268465e-01
[151] 9.691538e-01 9.691538e-01 9.373228e-01 2.553160e-01 2.259326e-01
[156] 1.625547e-01 3.104198e-01 8.454135e-01 2.604905e-01 9.043438e-01
[161] 2.632667e-01 4.206234e-01 9.691538e-01 2.958308e-02 6.436976e-01
[166] 1.042884e-01 4.251450e-04 9.421849e-01 4.326552e-02 8.454135e-01
[171] 4.820840e-01 5.044162e-02 6.847507e-01 7.694044e-01 4.820840e-01
[176] 7.468609e-01 6.608883e-01 7.312505e-01 3.844849e-01 3.471163e-01
[181] 5.494725e-01 5.659931e-01 6.859654e-01 4.291281e-01 6.330223e-01
[186] 1.911166e-01 8.709233e-01 2.123658e-01 6.330223e-01 8.629376e-01
[191] 1.179944e-01 2.297710e-02 3.709659e-05 8.268199e-01 5.710903e-01
[196] 9.458067e-08 3.986278e-05 2.773402e-03 4.182808e-06 2.214614e-01
[201] 3.980075e-01 5.264525e-01 5.659931e-01 2.091344e-03 6.450831e-01
[206] 2.976282e-01 1.324357e-01 8.542679e-02 7.587533e-04 1.327839e-01
[211] 1.596949e-02 8.261010e-01 9.788828e-01 8.328305e-01 9.166887e-01
[216] 7.694044e-01 6.330223e-01 2.259326e-01 7.312505e-01 5.508422e-01
[221] 8.386332e-02 5.862159e-01 9.029180e-01 8.454135e-01 9.902980e-01
[226] 9.691538e-01 9.691538e-01 8.468829e-03 4.206234e-01 5.075209e-03
[231] 3.131189e-01 9.280934e-02 4.365918e-01 7.443624e-02 9.029180e-01
[236] 8.138160e-04 7.419157e-01 2.880934e-02 6.436976e-01 2.271639e-01
[241] 3.844849e-01 9.088553e-02 8.656084e-01 1.289511e-03 8.263821e-01
[246] 1.728407e-02 3.851312e-02 2.567175e-02 2.567175e-02 1.595265e-04
[251] 5.597545e-01 8.060503e-01 1.663355e-01 8.146607e-01 6.887757e-01
[256] 4.370757e-02 1.793697e-02 8.463109e-01 4.370757e-02 7.312505e-01
[261] 8.656084e-01 9.804359e-01 5.966002e-01 9.902980e-01 9.942214e-01
[266] 7.861626e-01 7.558959e-01 8.722588e-02 5.322309e-01 9.126008e-01
[271] 7.810493e-01 5.769588e-01 6.330223e-01 7.973155e-01 3.990947e-01
[276] 6.859654e-01 9.029180e-01 2.396694e-01 7.065366e-01 5.659931e-01
[281] 5.033219e-01 8.40070

define p-value cutoff: length(n) shows how many pairs will be retrieved with the defined cutoff

In [22]:
n <- which(sum_cont$p_value < 0.03)
length(n)

[1] 77

In [23]:
mypath <- c()
for (i in 1:length(n)){
  mID <- as.character(sum_cont[n[i],1])
  f <- as.character(sum_cont[n[i],2])

  b <- data.frame(metadata=metadata[,which(colnames(metadata)==mID)],feature=as.factor(data_cont[,which(colnames(data_cont)==f)]))
  b$det <- as.numeric(b$feature)
  b$det[which(b$feature=="1")] <- "detected"
  b$det[which(b$feature=="0")] <- "not detected"
  b$det <- as.factor(b$det)
  files_cont=paste(gsub("[.]","",mID), gsub("[[:punct:][:space:]]", "", f), ".pdf", sep="")
  
  if (nchar(files_cont) > 50) {
    
    files_cont = paste(substr(files_cont, 1,20),".pdf",sep="")
    pdf(file=files_cont, width=3, height=2)
    p<-ggplot(b, aes(x=det, y=metadata, color=det)) + geom_boxplot(color=c("#F8766D","#00BFC4")) + labs(x = f ,y=mID,cex=5) + theme(axis.title=element_text(size=7),axis.text =element_text(size=5), ) + coord_flip()
    print(p)
    dev.off()
    mypath <- c(mypath,files_cont)
    
    } else {
      
      pdf(file=files_cont, width=3, height=3)
      p<-ggplot(b, aes(x=det, y=metadata, color=det)) + geom_boxplot(color=c("#F8766D","#00BFC4")) + labs(x = f ,y=mID,cex=5) + theme(axis.title=element_text(size=7),axis.text =element_text(size=5), ) + coord_flip()
      print(p)
      dev.off()
      mypath <- c(mypath,files_cont)
      
    }
}

sum_cont$plot <- "none"
sum_cont$plot[n] <- paste0('<a href=\"',"file://",mypath,'">plot</a>')

write.csv(sum_cont, "DifferentialassociationTesting_MS2/SummaryTable_continuous.csv",quote=F)

dt <- DT::datatable(sum_cont[n,], selection =  list( mode = 'single'),escape=F)
saveWidget(dt, file="SummaryTable_continuous.html")

In [27]:
write.table(sum_cont, "DifferentialassociationTesting_MS2/SummaryTable_continuous.txt", sep="\t")